In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.stattools as ts
from statsmodels.iolib.summary2 import summary_col

### STARGAZER ###

from stargazer.stargazer import Stargazer, LineLocation

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/df_nzgas.csv'
df = pd.read_csv(url)
df["DATE"] = pd.to_datetime(df["DATE"],dayfirst=True)
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col],errors='coerce')

df["nzbtuusd_r"] = (df["NZBTU_USD"].pct_change())
df = df.dropna()
print(df)

         DATE  Price_NZ    NZ_BTU  NZ_USD_Q  NZBTU_USD        HH  nzbtuusd_r
1  1999-06-01  2.936804  3.098492  0.537400   1.665130  2.290000    0.013934
2  1999-09-01  2.698549  2.847121  0.521433   1.484584  2.693333   -0.108428
3  1999-12-01  2.827617  2.983294  0.511567   1.526154  2.383333    0.028001
4  2000-03-01  2.871244  3.029323  0.493033   1.493557  2.830000   -0.021359
5  2000-06-01  2.719361  2.869078  0.467333   1.340816  3.956667   -0.102267
..        ...       ...       ...       ...        ...       ...         ...
86 2020-09-01  6.594163  6.957211  0.663333   4.614950  2.203333    0.093243
87 2020-12-01  6.825113  7.200877  0.704467   5.072778  2.636667    0.099205
88 2021-03-01  7.414286  7.822487  0.717100   5.609505  3.543333    0.105805
89 2021-06-01  9.099814  9.600814  0.710567   6.822018  3.336667    0.216153
90 2021-09-01  9.154261  9.658258  0.702267   6.782672  4.913333   -0.005767

[90 rows x 7 columns]


In [ ]:
mod = sm.tsa.MarkovRegression(df["nzbtuusd_r"], k_regimes=2, switching_variance = True)
res_mod = mod.fit()
res_mod.summary()

In [ ]:
df['ratio'] = df["HH"]/df["NZBTU_USD"]
plt.plot(df['DATE'],df['ratio'],color='blue')

In [ ]:
mod = sm.tsa.MarkovRegression(df["ratio"], k_regimes=2, switching_variance = True)
res_mod = mod.fit()
res_mod.summary()

In [ ]:
mod_1 = sm.tsa.MarkovRegression(df["ratio"], k_regimes=2, switching_variance = True)
res_mod_1 = mod_1.fit()
mod_2 = sm.tsa.MarkovRegression(df["ratio"], k_regimes=3, switching_variance = True)
res_mod_2 = mod_2.fit()
mod_3 = sm.tsa.MarkovRegression(df["ratio"], k_regimes=4, switching_variance = True)
res_mod_3 = mod_3.fit()

In [ ]:
res_mod.filtered_marginal_probabilities[1].plot(
    title='Probability of being in high price/volatility regime', figsize=(12,2))
plt.savefig('/Users/joe/Dropbox/PhD/Writing/TeX/Charts/nzhhmarkovprob.png', dpi=400)

In [ ]:
print(res_mod.summary().as_latex())

In [1]:
## example code for latex summary_col([reg0,reg1,reg2],stars=True,float_format='%0.2f') ##

ModuleNotFoundError: No module named 'statsmodels.tsa.vecm'